In [16]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2 as pg
import pandas.io.sql as psql
import geopandas as gpd
# import folium
# from folium import FeatureGroup, LayerControl, Map, Marker
import pymongo
import numpy as np

In [17]:
#connect with postgresql and extract data (4 tables)
connection = pg.connect("host=localhost dbname=project2 user=postgres password=ynot8888")
export_df = psql.read_sql('SELECT * FROM export', connection)

In [18]:
export_df.head()

,id,countries,y_1987,y_1988,y_1989,y_1990,y_1991,y_1992,y_1993,y_1994,...,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,China ...,1350.175,1462.099,1485.165,1606.502,1645.346,2518.077,2910.378,3338.666,...,71485.71636,80623.01946,79164.34138,99985.835440,85070.360070,87830.07330,110029.94580,123603.50880,153515.28550,167625.45500
1,2,Japan ...,12133.279,13672.807,14617.324,16599.460,17284.284,18157.398,19059.231,19706.973,...,48950.42016,53183.34539,48540.45947,50972.119240,46443.689700,37974.17131,44455.81339,51256.45539,61675.49188,56161.03845
2,3,Republic of Korea ...,1877.400,2313.136,2829.305,3412.017,3594.093,4328.702,5242.378,6084.613,...,24583.22727,24061.32650,21091.57327,22667.692170,20613.394500,19970.71823,22870.59425,23713.18440,27838.57754,27647.13009
3,4,United States ...,6081.367,6128.277,7081.439,7782.911,7097.017,7067.138,7471.259,7045.994,...,14765.82122,15577.30687,15054.67683,17228.086350,20528.413500,21896.23918,20694.83201,21354.73044,24768.59842,27403.93459
4,5,United Kingdom ...,2987.145,3020.324,3305.382,3494.435,3801.413,4167.042,4732.689,4197.260,...,11099.92775,12422.80980,10161.41093,8392.474061,9095.184962,12641.40766,12753.82049,11758.70499,13450.23836,20950.50024


In [19]:
export_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         229 non-null    int64  
 1   countries  229 non-null    object 
 2   y_1987     229 non-null    float64
 3   y_1988     229 non-null    float64
 4   y_1989     229 non-null    float64
 5   y_1990     229 non-null    float64
 6   y_1991     229 non-null    float64
 7   y_1992     229 non-null    float64
 8   y_1993     229 non-null    float64
 9   y_1994     229 non-null    float64
 10  y_1995     229 non-null    float64
 11  y_1996     229 non-null    float64
 12  y_1997     229 non-null    float64
 13  y_1998     229 non-null    float64
 14  y_1999     229 non-null    float64
 15  y_2000     229 non-null    float64
 16  y_2001     229 non-null    float64
 17  y_2002     229 non-null    float64
 18  y_2003     229 non-null    float64
 19  y_2004     229 non-null    float64
 20  y_2005    

In [20]:
import_df = psql.read_sql('SELECT * FROM import', connection)
import_df.head()

,id,countries,y_1987,y_1988,y_1989,y_1990,y_1991,y_1992,y_1993,y_1994,...,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,China ...,834.271,996.650,1355.004,1456.049910,1864.599830,2549.469540,3225.457082,3745.017037,...,40923.57616,43919.33875,46442.35528,49570.57342,54460.00207,64377.15146,62323.16915,67448.49916,78303.08758,82841.71216
1,2,United States ...,10820.323,12121.967,15206.514,15983.179800,15816.686180,16390.622110,17301.426170,19018.727460,...,36378.27321,40711.59793,43310.11822,41003.49267,43391.76404,49372.25903,44953.56040,47331.93812,50864.93788,55532.51401
2,3,Japan ...,8605.886,9601.499,11768.902,10641.232950,10018.172420,11377.087870,13263.953010,13649.569670,...,20573.47864,20078.83661,21420.45276,21608.57835,20095.80704,23027.15126,22823.68513,24812.99296,26987.55465,26357.17838
3,4,Germany ...,3130.720,3215.265,3739.115,3828.421334,3584.857050,3706.086275,4042.043774,4540.279529,...,12640.02702,12677.91634,13402.83038,14132.66079,14742.82575,15569.77274,16471.82198,16991.89263,18771.63892,18183.63119
4,5,Thailand ...,509.124,596.471,822.311,852.803585,887.411213,1054.844332,1133.793313,1240.562550,...,13265.71634,11065.98420,13084.97606,14287.43509,13266.02842,15804.57262,16878.39767,17213.32728,17913.66704,17385.42425


In [21]:
importtop25_df = psql.read_sql('SELECT * FROM top25import', connection)
importtop25_df.head(30)

,id,commodities,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,Personal travel (excl education) services ...,36426.000,37731.000,42684.000,46321.000,33288.000
1,2,Refined petroleum ...,16227.992,17389.114,21687.610,25069.399,21720.734
2,3,Passenger motor vehicles ...,21541.778,21781.807,23299.388,21573.858,19092.510
3,4,Telecom equipment & parts ...,11781.035,11969.132,13412.081,14590.189,15230.420
4,5,Computers ...,7819.856,7853.593,8836.896,9763.131,10398.009
5,6,Freight services ...,9866.000,8429.000,9428.000,10114.000,10363.000
6,7,Crude petroleum ...,7624.600,8573.768,11738.091,13412.252,9474.257
7,8,Gold ...,6044.777,7240.007,6540.415,5516.931,8811.644
8,9,Professional services ...,6008.000,6123.000,6657.000,7718.000,8291.000
9,10,Medicaments (incl veterinary) ...,7711.649,7339.659,7168.922,7482.139,8123.572


In [22]:
exporttop25_df = psql.read_sql('SELECT * FROM top25export', connection)
exporttop25_df.head(30)

,id,commodities,y_2015,y_2016,y_2017,y_2018,y_2019
0,1,Iron ores & concentrates ...,47798.926,62617.201,61392.008,77519.590,102863.856
1,2,Coal ...,34541.191,54236.174,60378.810,69595.465,54620.426
2,3,Natural gas ...,16575.582,22307.902,30906.613,49727.233,47524.587
3,4,Education-related travel services (c) ...,24145.000,28093.000,32602.000,37824.000,39661.000
4,5,Gold ...,16585.176,18978.819,19292.604,18866.943,24393.843
5,6,Personal travel (excl education) services ...,20669.000,21628.000,21332.000,22450.000,16368.000
6,7,"Beef, f.c.f. ...",8494.648,7114.836,7962.722,9475.854,11257.781
7,8,Aluminium ores & conc (incl alumina) ...,6789.681,7529.116,9447.839,11357.604,8875.375
8,9,Crude petroleum ...,5184.027,5150.102,6505.871,8494.126,8568.158
9,10,Copper ores & concentrates ...,4664.019,4577.348,5700.413,5968.546,6854.001


In [23]:
#read in geojson file and convert into df
fname = "csv&geojson/countries.geojson"

gjs_df = gpd.read_file(fname)
gjs_df.head()

,ADMIN,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [24]:
gjs_df.insert(loc=1, column='year', value=['' for i in range(gjs_df.shape[0])])

In [25]:
gjs_df.head()

,ADMIN,year,ISO_A3,geometry
0,Aruba,,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [26]:
gjs_df.insert(loc=2, column='value', value=['' for i in range(gjs_df.shape[0])])
gjs_df.head()

,ADMIN,year,value,ISO_A3,geometry
0,Aruba,,,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,,,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,,,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,,,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,,,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [27]:
gjs_df.to_file("csv&geojson/mycountries.geojson", driver="GeoJSON")

In [28]:
y1987export_df = export_df[["countries","y_1987" ]].copy()
y1987export_df.head()

,countries,y_1987
0,China ...,1350.175
1,Japan ...,12133.279
2,Republic of Korea ...,1877.400
3,United States ...,6081.367
4,United Kingdom ...,2987.145


In [29]:
sorted_df = y1987export_df.sort_values(by=['y_1987'], ascending=False)
cutted_df = sorted_df.iloc[:30, :]
# cutted_df.head(30)
# cutted_df.countries.count()

In [30]:
y1987 = []

for i in range(0, 30):
    temp = {}
    temp["country"] = cutted_df.iloc[i, 0] 
    temp["value"] = cutted_df.iloc[i, 1]
    y1987.append(temp)
print(y1987)    

[{'country': 'Japan                                             ', 'value': 12133.279}, {'country': 'United States                                     ', 'value': 6081.367}, {'country': 'New Zealand                                       ', 'value': 3326.289}, {'country': 'United Kingdom                                    ', 'value': 2987.145}, {'country': 'Hong Kong (SAR of China)                          ', 'value': 2267.35}, {'country': 'Republic of Korea                                 ', 'value': 1877.4}, {'country': 'Singapore                                         ', 'value': 1472.861}, {'country': 'Taiwan                                            ', 'value': 1447.482}, {'country': 'China                                             ', 'value': 1350.175}, {'country': 'Germany                                           ', 'value': 1349.694}, {'country': 'Italy (e)                                         ', 'value': 1166.985}, {'country': 'France (d)                                

In [31]:
ecolumn_list = [c for c in export_df.columns]
ecol_use_list = ecolumn_list[2:]
print(ecol_use_list)

['y_1987', 'y_1988', 'y_1989', 'y_1990', 'y_1991', 'y_1992', 'y_1993', 'y_1994', 'y_1995', 'y_1996', 'y_1997', 'y_1998', 'y_1999', 'y_2000', 'y_2001', 'y_2002', 'y_2003', 'y_2004', 'y_2005', 'y_2006', 'y_2007', 'y_2008', 'y_2009', 'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [32]:
#To build MongoDB and store the DATA
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.project2DB

In [65]:
exports = db.exports
imports = db.imports
top25exports = db.top25exports
top25imports = db.top25imports

In [33]:
completeexports = db.completeexports
completeimports = db.completeimports

In [97]:
#db.exports.delete_many({})
#db.imports.delete_many({})
# db.top25exports.delete_many({})
# db.top25imports.delete_many({})

In [35]:
print(len(export_df["countries"]))

229


In [37]:
# build json file for db.completeexports

for i in ecol_use_list:
     
    temp_list =[]

    temp_df = export_df[["countries", i ]].copy()
    
    
#     print(stcutted_df.head())
    
    for j in range(0, len(temp_df["countries"])):
        temp_dict = {}
        temp_dict["country"] = temp_df.iloc[j, 0] 
        temp_dict["value"] = temp_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty    
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.completeexports.insert_one(bag)
       

In [38]:
for i in ecol_use_list:
     
    temp_list =[]

    temp_df = import_df[["countries", i ]].copy()
    
    
#     print(stcutted_df.head())
    
    for j in range(0, len(temp_df["countries"])):
        temp_dict = {}
        temp_dict["country"] = temp_df.iloc[j, 0] 
        temp_dict["value"] = temp_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty    
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.completeimports.insert_one(bag)
       

In [74]:
# build json file for db.exports

for i in ecol_use_list:
     
    temp_list =[]

    temp_df = export_df[["countries", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
    stcutted_df = sorted_df.iloc[:30, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 30):
        temp_dict = {}
        temp_dict["country"] = stcutted_df.iloc[j, 0] 
        temp_dict["value"] = stcutted_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.exports.insert_one(bag)
    
    

In [79]:
# import_df columns' name list
icolumn_list = [c for c in import_df.columns]
icol_use_list = icolumn_list[2:]
print(icol_use_list)

['y_1987', 'y_1988', 'y_1989', 'y_1990', 'y_1991', 'y_1992', 'y_1993', 'y_1994', 'y_1995', 'y_1996', 'y_1997', 'y_1998', 'y_1999', 'y_2000', 'y_2001', 'y_2002', 'y_2003', 'y_2004', 'y_2005', 'y_2006', 'y_2007', 'y_2008', 'y_2009', 'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [80]:
# build json file for db.imports

for i in icol_use_list:
     
    temp_list =[]

    temp_df = import_df[["countries", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
    stcutted_df = sorted_df.iloc[:30, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 30):
        temp_dict = {}
        temp_dict["country"] = stcutted_df.iloc[j, 0] 
        temp_dict["value"] = stcutted_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.imports.insert_one(bag)  

In [83]:
# importtop25_df columns' name list
it25column_list = [c for c in importtop25_df.columns]
it25col_use_list = it25column_list[2:]
print(it25col_use_list)

['y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [99]:
# build json file for db.top25imports

for i in it25col_use_list:
     
    temp_list =[]

    temp_df = importtop25_df[["commodities", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
#     stcutted_df = sorted_df.iloc[:25, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 25):
        temp_dict = {}
        temp_dict["commodities"] = sorted_df.iloc[j, 0] 
        temp_dict["value"] = sorted_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.top25imports.insert_one(bag)  

In [100]:
# exporttop25_df columns' name list
et25column_list = [c for c in importtop25_df.columns]
et25col_use_list = et25column_list[2:]
print(et25col_use_list)

['y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019']


In [101]:
# build json file for db.top25exports

for i in et25col_use_list:
     
    temp_list =[]

    temp_df = exporttop25_df[["commodities", i ]].copy()
    sorted_df = temp_df.sort_values(by=[i], ascending=False)
#     stcutted_df = sorted_df.iloc[:25, :]
    
#     print(stcutted_df.head())
    
    for j in range(0, 25):
        temp_dict = {}
        temp_dict["commodities"] = sorted_df.iloc[j, 0] 
        temp_dict["value"] = sorted_df.iloc[j, 1]
    
        temp_list.append(temp_dict)
             
    temp_df.empty
    sorted_df.empty
    stcutted_df.empty
    
#     print(temp_list)
     
    bag = {}
    bag[i] = temp_list
    db.top25exports.insert_one(bag)  

In [42]:
n = db.completeimports.count_documents({}) 
print (n)       

33


In [43]:
exports = db.completeimports.find()
for i in exports:
    print(i)

{'_id': ObjectId('6080d3f3ef4f4d2764e849f5'), 'y_1987': [{'country': 'China                                             ', 'value': 834.271}, {'country': 'United States                                     ', 'value': 10820.323}, {'country': 'Japan                                             ', 'value': 8605.886}, {'country': 'Germany                                           ', 'value': 3130.72}, {'country': 'Thailand                                          ', 'value': 509.124}, {'country': 'United Kingdom                                    ', 'value': 5091.309}, {'country': 'Singapore                                         ', 'value': 1335.48}, {'country': 'New Zealand                                       ', 'value': 2246.191}, {'country': 'Republic of Korea                                 ', 'value': 1069.689}, {'country': 'Malaysia                                          ', 'value': 648.926}, {'country': 'Italy (d)                                         ', 'value': 1534.915}, {